# DDL e DML

O SQL usa os termos **tabela**, **linha** e **coluna** para os termos do modelo relacional formal *relação*, *tupla* e *atributo* respectivamente.

In [1]:
# Utilizaremos a lib csvms em nossos exemplos
from csvms.table import Table

## Criação

O comando `CREATE TABLE` é usado para especificar uma nova relação, dando-lhe um **nome** e especificando seus **atributos**. Cada um dos atributos recebe um **nome** e um **tipo de dado** para especificar seu domínio de valores.

```sql
CREATE TABLE lista_frutas (
    nm_fruta TEXT ,
    tp_fruta TEXT )
```

> Como nosso banco de dados será escrito na linguagem [python](https://www.w3schools.com/python/default.asp), utilizaremos os seus [tipos primitivos](https://www.w3schools.com/python/python_datatypes.asp) como tipos de dados.

Os tipos de dados básicos disponíveis para atributos costumam ser:

+ Numéricos
+ Cadeia de caracters
+ Cadeia de bits
+ Booleano
+ Data
+ Hora

> E é já a partir deste ponto em que os vários dialetos começam a surgir...

![create table](../imgs/create_table.png)

In [2]:
# Criação
Table(
    # Nome da tabela
    name="lista_frutas",
    # Definição das colunas informadas
    columns={
        'nm_fruta':str, 
        'tp_fruta':str 
    }
# Salava a nova tabela para que ela seja salva no sistema de arquivos
).save()

True

## Transação

O inicio do trabalho `BEGIN` se dá com o carregamento da tabela do sistema de arquivos para a SGA (System Global Area).
Tudo o que ocorrer com este objeto durante seu período em mória será considerado uma transação até que se confirme com os comandos de `COMMIT` ou `ROLLBACK`

In [3]:
# Como só temos uma tabela envolvida nesta transação
# Salvaremos tudo o que acontece durannte o trabalho em uma variável
tbl=Table("lista_frutas")

## Inclusão

Em sua forma mais simples, o comando `INSERT` é usado para acrescentar uma única *tupla* (linha) em uma *relação* (tabela).
Precisamos especificar o nome da relação e uma lista de valores para a tupla. Os valores devem ser informados na mesma ordem em que os atributos correspondentes foram especificados no `CREATE TABLE`.

![insert](../imgs/insert.png)

```sql
INSERT INTO lista_frutas VALUES ('banana','doce');
INSERT INTO lista_frutas VALUES ('limão','azedo');
INSERT INTO lista_frutas VALUES ('maçã','doce');
```

In [4]:
# Cada um dos comandos de insert são convertidos em um append 
# na tabela da transação corrente
tbl.append('banana','doce')
tbl.append('limão','azedo')
tbl.append('maçã','doce')
# Vamos ver como ficou
print(tbl)


2022-07-26 15:44:30,346: Row ('banana', 'doce') inserted
2022-07-26 15:44:30,347: Row ('limão', 'azedo') inserted
2022-07-26 15:44:30,348: Row ('maçã', 'doce') inserted


TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|     doce|
  1|    limão|    azedo|
  2|     maçã|     doce|
   +---------+---------+



## Confirmação

Com o comando de `COMMIT` confirmamos que todas as alterações desta transação deverão ser **salvas** em definitivo.

```sql
COMMIT
```

In [5]:
# Como todas as modificações executadas estão dentro
# da variável, utiliza o comando save para persistí-las
tbl.save()
# Agora que a tabela foi salva, os dados nela contidos devem estar no arquivo csv dentro do seu diretório data/default

True

## Atualização

O comando `UPDATE` é utilizado para modificar valores de atributos de uma ou mais *tuplas* selecionadas. Assim como no comando `DELETE`, é possível adicionar uma clausula `WHERE` no comando para informar o critério de seleção das linhas que serão afetadas.

```sql
UPDATE lista_frutas 
   SET tp_fruta = 'amargo'
 WHERE tp_fruta = 'doce'
```

In [6]:
# Procura em toda a tabela por linhas cujo nome da fruta seja "limão"
# Itera sobre a tabela para percorrer todas as sua linha
for idx in range(len(tbl)):
    # em busca daquelas que atendem às condições da clausula WHERE
    if tbl[idx]["tp_fruta"]=='doce':
        # Atribui o conteúdo da linha encontrada a uma variável
        row = tbl[idx]
        # para aplicar o 'SET' definido pelo usuário
        # alterando seu conteúdo
        row["tp_fruta"]='amargo'
        # Atualiza a linha da tabela com a nova tupla
        tbl[idx] = tuple(row.values())
# Podemos ver como os dados ficaram imprimindo a variável que contem a tabela
print(tbl)

2022-07-26 15:44:30,453: Row 0 updated with values ('banana', 'amargo')
2022-07-26 15:44:30,455: Row 2 updated with values ('maçã', 'amargo')


TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|   amargo|
  1|    limão|    azedo|
  2|     maçã|   amargo|
   +---------+---------+



## Exclusão

O comando `DELETE` remove *tuplas* de uma relação. Ele possui uma clausula *`WHERE`*, semelhante a utilzada em consultas, para selecionar a tuplas que serão excluídas.

```sql
DELETE FROM lista_frutas WHERE nm_fruta = 'limão'
```

In [7]:
# Precisamos iterar sobre as linhas da tabela
# A iteração ocorre de forma reversa por que o comando
# del pode alterar seus indices
for idx in reversed(range(len(tbl))): #As linhas estão sendo analisadas em ordem reversa por que os indices podem alterar ao remover mais de 1 linha
    # Para descobri qual delas atendentem às condições
    # especificadas pelo usuário na cláusula WHERE
    if tbl[idx]["nm_fruta"]=="limão":
        # Quando encontrar uma linha que passe na condição
        # Remove a linha da tabela com o comando del
        del tbl[idx]

print(tbl)


2022-07-26 15:44:30,503: Row ('limão', 'azedo') deleted


TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|   amargo|
  1|     maçã|   amargo|
   +---------+---------+



## Descarte

Com o comando de `ROLLBACK` descartamos todas as alterações desta transação, voltando assim ao estado em que os dados se encontravam desde a ultima operação salva. O comando de `ROLLBACK` finalizará um trabalho e iniciará outro.

```sql
ROLLBACK
```

In [8]:
# Dado que todas as modificações feitas nessa transação estão contidas na variável
# Ao sobreescrevê-la com o conteúdo do arquivo estamos "jogando-as fora"
# e retornando aos valores desde o ultimo save (COMMIT)
tbl = Table('lista_frutas')

print(tbl)


TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|     doce|
  1|    limão|    azedo|
  2|     maçã|     doce|
   +---------+---------+



## Alteração

O comando `ALTER TABLE` serve para alterar os atributos definidos para a tabela durante o processo de criação `CREATE TABLE`. Seus principais operadores servem para adição (`ADD`), modificação (`MODIFY`) tipo de dado, renomear (`RENAME`) um atributo ou ainda a remoção (`DROP`) de atributos existentes.

> Este sistema não prevê *concorrencia*, por isso podemos trabalhar sobre os objetos da SGA

```sql
ALTER TABLE lista_frutas ADD (valor INTEGER)
```

In [9]:
# Adiciona uma nova coluna do tipo inteiro chamada valor
tbl.alter('ADD',{'valor':int})
print(tbl)

TABLE: default.lista_frutas
   +---------+---------+------+
   |nm_fruta |tp_fruta |valor |
   +---------+---------+------+
  0|   banana|     doce|     0|
  1|    limão|    azedo|     0|
  2|     maçã|     doce|     0|
   +---------+---------+------+



```sql 
ALTER TABLE lista_frutas MODIFY (valor INTEGER) TO (vl_fruta FLOAT);
```

In [10]:
# Renomear a coluna para a convenção de nomes utilizada na tabela e alterar seu tipo para ponto flutuante
tbl.alter('MODIFY',{'valor':int},{'vl_fruta':float})
print(tbl)

TABLE: default.lista_frutas
   +---------+---------+---------+
   |nm_fruta |tp_fruta |vl_fruta |
   +---------+---------+---------+
  0|   banana|     doce|      0.0|
  1|    limão|    azedo|      0.0|
  2|     maçã|     doce|      0.0|
   +---------+---------+---------+



```sql
ALTER TABLE lista_frutas DROP vl_fruta;
```

In [11]:
# Remove a coluna 'vl_fruta' da tabela
tbl.alter('DROP',{'vl_fruta':float})
print(tbl)

TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|     doce|
  1|    limão|    azedo|
  2|     maçã|     doce|
   +---------+---------+



## Eliminação

Como o nome diz, o comando `DROP TABLE` eliminará o objeto do sistema. Ou seja, tanto o que está em memória quanto no sistema de arquivos e catalogo de dados.

![drop table](../imgs/drop_table.png)

In [12]:
# Eliminar a tabela do sistema
tbl.drop()
# o DROP vai eliminar os arquivos fisicamente do sistema de arquivos

True